In [134]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from operator import itemgetter
import matplotlib as mpl
import matplotlib.pyplot as plt
from timeit import time
import itertools # para generar las permutaciones del exhaustivo
import matplotlib.pyplot as plt

capitales = pd.read_csv('capitales.csv')

# ------------------------ formateo de tabla de distancias
tabla_distancias = pd.read_csv('distancias_capitales.csv')
tabla_distancias = tabla_distancias.rename(columns={'Distancias en kilómetros': 'Capital'})
columnas = ['capital']
for i in range(24):
    columnas.append(i)
tabla_distancias.columns = columnas
# ------------------------

In [30]:
capitales=pd.read_excel('CapitalesArg.xlsx', index_col=None, header=None)
a=[]
a=list(capitales[1][1:24])
b=[]
for i in range(len(a)):
    if type(a[i])!=float:
        b.append(int(a[i]))
print(b[:])
print(str(np.min(b)))
min=str(np.min(b))
recorrido=[]
capis=capitales[1][1:24]
for id, cap in enumerate (capis):
                if cap==min:
                    recorrido.append(str(id+1))
print(recorrido)

[646, 792, 933, 53, 986, 985, 989, 375, 834, 1127, 794, 2082, 979, 1080, 1334, 1282, 1005, 749, 393, 579, 939, 2373]
53
['5']


In [194]:
def calcular_distancia(tabla_distancias, recorrido):
    '''
    dado un recorrido [ciudad 1, ciudad 2,  .., ciudad n, ciudad 1], calcula la longitud del trayecto
    '''
    distancia_recorrida = 0
    for i in range(len(recorrido) - 1):
        distancia_actual = tabla_distancias[recorrido[i]][recorrido[i+1]]
        distancia_recorrida += distancia_actual
    return distancia_recorrida

In [ ]:
#esta mal
def heuristicoA(tabla_capitales, nro_ciudad):
    recorrido=[]
    recorrido.append(str(nro_ciudad))
    elegida=nro_ciudad
    for nro_capi in range (1, 22):
        band=False
        while (band==False):
            a, b=[],[]
            a=list(tabla_capitales[elegida][1:24])
            for i in range(len(a)):
                if type(a[i])!=float:
                    b.append(int(a[i]))
            min=str(np.min(b))
            for id, cap in enumerate (tabla_capitales):
                if cap==min:
                    recorrido.append(str(id+1))
            #band=True
            for i in range(len(recorrido)-1):
                if i==len(recorrido)-2:
                    band=True
                if recorrido[i]==recorrido[-1]:
                    recorrido.pop()
                    band=False
                    break
        elegida=int(recorrido[-1])            
    recorrido.append(recorrido[0])
    return recorrido

In [125]:
def heuristicoA(tabla_distancias, nro_ciudad):
    recorrido = []
    recorrido.append(nro_ciudad)
    distancia_recorrida = 0
    
    id_capital_sig = nro_ciudad #
    tabla_auxiliar = tabla_distancias[id_capital_sig].copy() #inicializar tabla auxiliar

    while len(recorrido) < len(tabla_distancias):  # len(tabla_distancias) -> 24
    
        distancia_mas_cercana = tabla_auxiliar.min()
        id_capital_sig = tabla_auxiliar.idxmin()
        
        if id_capital_sig in recorrido:  # si verdadero, esa capital ya se visitó y por lo tanto se elimina de tabla auxiliar
            tabla_auxiliar = tabla_auxiliar.drop(id_capital_sig) 
        else:
            distancia_recorrida += distancia_mas_cercana  
            recorrido.append(id_capital_sig)
            tabla_auxiliar = tabla_distancias[id_capital_sig].copy() # reinicio la tabla auxiliar para que tenga todas las filas

    return recorrido, distancia_recorrida

In [2]:
def formatear(tabla_capitales, recorrido):
    # devuelve una tabla con latitudes y longitudes, ordenada por orden de recorrido lista para ser impresa en el mapa
    tabla_final = pd.DataFrame(columns=['capital', 'latitud', 'longitud'])
    
    for i in range(len(recorrido)):
        tabla_final = tabla_final.append(tabla_capitales.iloc[recorrido[i]])
        
    return tabla_final

In [131]:
# para probar heuristicoA
recorrido, distancia_recorrida = heuristicoA(tabla_distancias, 5)
s = formatear(capitales, recorrido)
s

,capital,latitud,longitud
5,La Rioja,-29.416018,-66.872266
13,S.F.d.V.d. Catamarca,-28.470552,-65.785805
21,Sgo. del Estero,-27.803519,-64.268081
14,S.M. de Tucumán,-26.824140,-65.222600
16,Salta,-24.793299,-65.413140
15,S.S. de Jujuy,-24.192795,-65.298883
11,Resistencia,-27.426056,-58.988848
2,Corrientes,-27.483745,-58.812996
3,Formosa,-26.181290,-58.190586
9,Posadas,-27.382250,-55.920130


In [135]:
s.iloc[0]['capital']

'La Rioja'

In [ ]:
#no esta probado
def heuristicoB(tabla_capitales):
    recorridos=[]
    for i in range (24):
        recorridos.append(heuristicoA(tabla_capitales, i+1))
    longitudes=[]
    for cromosoma in recorridos:
        id=0
        for prov in cromosoma:
            if id<23:
                kms=int(tabla_capitales[int(prov)][int(cromosoma[id+1])])+kms
                id=id+1
        kms=kms+int(tabla_capitales[int(cromosoma[-2])][int(cromosoma[-1])])
        longitudes.append(kms)
    for id, cromosoma in recorridos:
        aux.append(cromosoma, longitides[id])
    aux = sorted(aux, key=itemgetter(1), reverse=True)
    mejor_recorrido=aux[0][0]
    kms=aux[0][1]
    return mejor_recorrido, kms

# con alg. geneticos

In [101]:
def inicializar(tamaño_poblacion, longitud_cromosoma):
    poblacion_inicial = []
    for i in range(tamaño_poblacion):
        num = []
        num.append(np.random.randint(1,24))
        for i in range(1, longitud_cromosoma-1):
            band=True
            while(band==True):
                prov=np.random.randint(1,24)              
                repetido=0
                for j in (num):
                    if prov==j:
                        repetido=repetido+1
                if repetido<1:
                    num.append(prov)
                    band=False
        num.append(num[0])
        poblacion_inicial.append(num)
    
    return poblacion_inicial

In [73]:
def mutacion(cromosoma):
    aux = cromosoma[:-1]  # aux es el cromosoma pero sin la vuelta a casa
    
    provA = np.random.randint(1, len(aux)-1)
    provB = np.random.randint(1, len(aux)-1)
    #no se si haria falta validar que no toque la misma provincia a permutar
    band=False
    while (band==False):
        if provA==provB:
            provB = np.random.randint(1, len(aux)-1)
        else:
            band=True
    A=aux[provA]
    B=aux[provB]
    aux[provA]=B
    aux[provB]=A
    
    aux.append(cromosoma[0]) # vuelvo a agregar la vuelta a casa
    return aux

In [123]:
def crossover_ciclico(padre1, padre2):
    
    hijo1 = []
    hijo2 = []
    for i in range(len(padre1)):
        hijo1.append(-1)
        hijo2.append(-1)
        
    # HIJO 1
    valor_inicial = padre1[0]
    hijo1[0] = valor_inicial
    gen = padre2[0]
 
    i = 1
    while True:
        if padre1[0] == padre2[0]:
            break
        if padre1[i] == gen:
            hijo1[i] = gen
            gen = padre2[i]
            if valor_inicial == padre2[i]:
                break
        if i >= len(padre1)-1:
            i = 1
        else:
            i += 1
    #intercambiar los que quedaron vacíos
    for i, prov in enumerate(hijo1):
        if prov == -1:
            hijo1[i] = padre2[i]
            
    # HIJO 2
    valor_inicial = padre2[0]
    hijo2[0] = valor_inicial
    gen = padre1[0]
 
    i = 1
    while True:
        if padre1[0] == padre2[0]:
            break
        if padre2[i] == gen:
            hijo2[i] = gen
            gen = padre1[i]
            if valor_inicial == padre1[i]:
                break
        if i >= len(padre1)-1:
            i = 1
        else:
            i += 1
    
    #intercambiar los que quedaron vacíos
    for i, prov in enumerate(hijo2):
        if prov == -1:
            hijo2[i] = padre1[i]
    
    return hijo1, hijo2

In [103]:
# PARA PROBAR EL CROSSOVER:
p1 = [9,8,2,1,7,4,5,10,6,3,9]  # con vuelta a casa
p2 = [9,2,3,4,5,6,7,8,1,10,9]  # con vuelta a casa
h1, h2 = crossover_ciclico(p1,p2)
print(h1) # hijos sin vuelta a casa
print(h2) # hijos sin vuelta a casa

[9, 2, 3, 4, 5, 6, 7, 8, 1, 10]
[9, 8, 2, 1, 7, 4, 5, 10, 6, 3]


In [ ]:
poblacion=inicializar(5, 24)
hijo1, hijo2= crossover_ciclico(poblacion[1], poblacion[3])
print(hijo1)
print(hijo2)
print(len(hijo1))

In [63]:
def func_objetivo(poblacion, distancias):
    f_objetivo=0
    #sumatoria de todas las distancias de todas las poblaciones
    for cromosoma in poblacion:
        id=0
        for prov in cromosoma:
            if id < len(cromosoma) - 1:
                f_objetivo=int(distancias[int(prov)][int(cromosoma[id+1])])+f_objetivo
                id=id+1
        f_objetivo=f_objetivo+int(distancias[int(cromosoma[-2])][int(cromosoma[-1])])          
    return f_objetivo

In [6]:
def func_objetivo_v2(poblacion, distancias):
    f_objetivo=0
    #sumatoria de todas las distancias de todas las poblaciones
    for cromosoma in poblacion:
        f_objetivo = f_objetivo + calcular_distancia(tabla_distancias, cromosoma)
    return f_objetivo

In [196]:
def fitness(cromosoma, f_obj, distancias):
    #fitness=[(fobj - recorrido) / fobj)]
    recorrido=0
    id=0
    for prov in cromosoma:
        if id < len(cromosoma) - 1:
            recorrido=int(distancias[int(prov)][int(cromosoma[id+1])])+recorrido
            id=id+1
    recorrido=recorrido+int(distancias[int(cromosoma[-2])][int(cromosoma[-1])])
    fitness=(f_obj-recorrido)/f_obj
    return fitness

In [7]:
def fitness_v2(poblacion, cromosoma, f_obj):
    recorrido = calcular_distancia(tabla_distancias, cromosoma)
    suma_complemento = 0
    
    for r in poblacion:
        suma_complemento = suma_complemento + (f_obj - calcular_distancia(tabla_distancias, r))
        
    fitness = (f_obj - recorrido) / suma_complemento
    return fitness

In [166]:
a=inicializar(5,24)
print(a[0])
f_obj=func_objetivo(inicializar(5, 24), capitales)
print(f_obj)
print(fitness(a[4], f_obj, capitales))

['18', '2', '19', '21', '15', '6', '13', '22', '20', '5', '23', '9', '11', '12', '4', '3', '17', '7', '10', '24', '16', '1', '8', '18']
139417
0.7989915146646391


In [187]:
# probar
def seleccionar(poblacion, distancias, elitismo = False):
    lista_poblacion = []
    id_seleccionado = 0
    padres = []
    fobj = func_objetivo_v2(poblacion, distancias)
    for id, individuo in enumerate(poblacion):
        lista_poblacion.append([id, individuo, fitness_v2(poblacion, individuo, fobj)])
    lista_poblacion = sorted(lista_poblacion, key=itemgetter(2), reverse=True)

    if elitismo:
        padres.append(lista_poblacion[0][1]) #los dos individuos elitistas
        padres.append(lista_poblacion[1][1])
    else:
        # RULETA 
        
        while len(padres) < 2:  # porque necesitamos dos padres
            num_aleatorio = np.random.rand()    
            fitness_acumulada = lista_poblacion[0][2]
            i = 0
            
            while(fitness_acumulada < 1):
                if num_aleatorio <= fitness_acumulada:
                    id_seleccionado = lista_poblacion[i][0]
                    break
                else: 
                    i += 1
                    fitness_acumulada += lista_poblacion[i][2]
                    
            if fitness_acumulada == 1:
                id_seleccionado = lista_poblacion[-1][0]

            for sublista in lista_poblacion:
                if sublista[0] == id_seleccionado:
                    cromosoma_elegido = sublista[1]
                    break
                    
            if cromosoma_elegido not in padres:
                padres.append(cromosoma_elegido)
                
    return padres[0], padres[1]

In [188]:
def generar_estadisticos(poblaciones, tabla_distancias):
    '''Genera la tabla, cada fila es una población'''
    resultados = [[],[],[],[],[]] # valor_fobjetivo, recorrido minimo, recorrido maximo, mediana recorridos, mejor cromosoma
    
    for poblacion in poblaciones:
        recorridos = []
        menor_recorrido = 999999999
        for cromosoma in poblacion:
            recorrido = calcular_distancia(tabla_distancias, cromosoma)
            recorridos.append((cromosoma, recorrido))   
            
            if recorrido < menor_recorrido:
                mejor_cromosoma = cromosoma
                menor_recorrido = recorrido
        recorridos = np.array(recorridos)

        resultados[0].append(np.sum(recorridos[:, 1]))
        resultados[1].append(np.min(recorridos[:, 1]))
        resultados[2].append(np.max(recorridos[:, 1]))
        resultados[3].append(np.median(recorridos[:, 1]))
        resultados[4].append(mejor_cromosoma)
        
    df = pd.DataFrame(resultados).transpose()
    df.columns = ['Función Objetivo Poblacion', 'Distancia mínima', 'Distancia máxima', 'Mediana de distancias', 'Mejor cromosoma']
    return df

In [189]:
#ver y/o arreglar generar_graficos()

In [195]:
# probar
def main(distancias, p_crossover=0.9, p_mutacion=0.2, ciclos=200, tamaño_poblacion=50, longitud_cromosoma=24, elitismo=True):
    t0 = time.perf_counter()
    
    poblacion_actual = inicializar(tamaño_poblacion, longitud_cromosoma)
    poblaciones = []
    poblaciones.append(poblacion_actual)
    cantidad_selecciones = tamaño_poblacion // 2

    for i in range(ciclos):
        print(f'ciclo {i}')
        poblacion_nueva = []

        for j in range(cantidad_selecciones + 1):
         
            if len(poblacion_nueva) == tamaño_poblacion:
                continue
                
            # SELECCIONAR
            padre_1, padre_2 = seleccionar(poblacion_actual, distancias, elitismo)
            
            # CRUZAR
            if np.random.rand() <= p_crossover:
                hijo_1, hijo_2 = crossover_ciclico(padre_1[:-1], padre_2[:-1])
                hijo_1.append(hijo_1[0])   # para que el hijo vuelva a casa
                hijo_2.append(hijo_2[0])   # para que el hijo vuelva a casa

            else: 
                hijo_1 = padre_1
                hijo_2 = padre_2

            # MUTAR
            if np.random.rand() <= p_mutacion:
                hijo_1 = mutacion(hijo_1)
            if np.random.rand() <= p_mutacion:
                hijo_2 = mutacion(hijo_2)

            # PASAR A PROXIMA GENERACION
            poblacion_nueva.append(hijo_1)
            if len(poblacion_nueva) != tamaño_poblacion:
                poblacion_nueva.append(hijo_2)

        poblacion_actual = poblacion_nueva
        
        poblaciones.append(poblacion_nueva)
        objetivos.append(func_objetivo_v2(poblacion_nueva, distancias))
    tf = time.perf_counter()
    
    poblaciones_df = pd.DataFrame(poblaciones)
    resultados = generar_estadisticos(poblaciones, tabla_distancias)
#    generar_graficos(resultados)
    tiempo_ejecucion = tf - t0
    
    return resultados, tiempo_ejecucion

In [196]:
resultados, tiempo = main(tabla_distancias, ciclos = 5, tamaño_poblacion=10)
resultados

ciclo 0
ciclo 1
ciclo 2
ciclo 3
ciclo 4


,Función Objetivo Poblacion,Distancia mínima,Distancia máxima,Mediana de distancias,Mejor cromosoma
0,271966,22512,29488,27947.5,"[22, 7, 6, 11, 17, 9, 1, 20, 8, 4, 10, 12, 5, ..."
1,245272,22512,28929,23843,"[22, 7, 6, 11, 17, 9, 1, 20, 8, 4, 10, 12, 5, ..."
2,244940,23843,25101,24866,"[22, 7, 6, 11, 17, 1, 8, 20, 9, 3, 10, 12, 5, ..."
3,237220,22636,24645,23843,"[22, 7, 6, 4, 17, 1, 8, 20, 9, 3, 10, 12, 5, 1..."
4,230084,22588,24516,22837,"[22, 7, 6, 4, 17, 1, 8, 20, 3, 9, 10, 12, 5, 1..."
5,227728,22588,23711,22636,"[22, 7, 6, 4, 17, 1, 8, 20, 3, 9, 10, 12, 5, 1..."


In [204]:
resultados.iloc[-1,:]

Función Objetivo Poblacion                                               227728
Distancia mínima                                                          22588
Distancia máxima                                                          23711
Mediana de distancias                                                     22636
Mejor cromosoma               [22, 7, 6, 4, 17, 1, 8, 20, 3, 9, 10, 12, 5, 1...
Name: 5, dtype: object

In [ ]:
# sin adaptar, solo sirve de ejemplo

def generar_graficos():
        
    # ----------------------------------------------------------------------------------------------
    # GENERACIÓN DE GRÁFICOS
    fig, (ax0, ax1, ax2) = plt.subplots(3, 1, figsize = (13, 8), sharex = True)

    ax0.plot(ids, medias, label='Media', linewidth=1.7, color="firebrick")
    ax1.plot(ids, minimos, label='Mínimo', linewidth=1.7, color="blue")
    ax2.plot(ids, maximos, label='Máximo', linewidth=1.7, color="green")

    ax0.set_title(f'Medias         ---         Valor final: {medias[-1]:2.4f}') 
    ax0.set_ylabel('Media')
    ax0.set_xlim(0,len(ids) - 1)
    ax0.grid(True)
    ax0.legend(loc='lower right')
    ax0.tick_params(axis='both', which='major', labelsize=12)
    ax0.tick_params(axis='both', which='minor', labelsize=12)
    
    ax1.set_title(f'Mínimos         ---         Valor final: {minimos[-1]:2.4f}') 
    ax1.set_ylabel('Mínimo')
    ax1.set_xlim(0,len(ids) - 1)
    ax1.grid(True)
    ax1.legend(loc='lower right')
    ax1.tick_params(axis='both', which='major', labelsize=12)
    ax1.tick_params(axis='both', which='minor', labelsize=12)
    
    
    ax2.set_title(f'Máximos         ---         Valor final: {maximos[-1]:2.4f}') 
    ax2.set_ylabel('Máximo')
    ax2.set_xlim(0,len(ids) - 1)
    ax2.set_xlabel('Número de generación')
    ax2.grid(True)
    ax2.legend(loc='lower right')
    ax2.tick_params(axis='both', which='major', labelsize=12)
    ax2.tick_params(axis='both', which='minor', labelsize=12)
    
    if ids[-1] <= 20:    
        ax2.set_xticks(ids)
    
    
    plt.rcParams.update({'font.size': 10})
    plt.tight_layout()
    
    
    plt.show()
    

# exhaustivo

In [8]:
def generar_recorridos_posibles(cantidad_ciudades):
    '''
    componente para el método exhaustivo. Genera todos los recorridos posibles de longitud cantidad_ciudades
    siempre partiendo desde Bs. As. (para simplificar)
    '''
    aux = list(itertools.permutations([i for i in range(cantidad_ciudades)])) # generar permutaciones de las capitales
    
    recorridos_posibles = []
    for recorrido in aux:
        recorrido_aux = list(recorrido)   # itertools devuelve tuplas pero lo paso a listas
        recorrido_aux.append(recorrido[0]) # para que vuelva a la ciudad de partida
        recorridos_posibles.append(recorrido_aux)
    return recorridos_posibles

In [9]:
def calcular_distancia(tabla_distancias, recorrido):
    '''
    dado un recorrido [ciudad 1, ciudad 2,  .., ciudad n, ciudad 1], calcula la longitud del trayecto
    '''
    distancia_recorrida = 0
    for i in range(len(recorrido) - 1):
        distancia_actual = tabla_distancias[recorrido[i]][recorrido[i+1]]
        distancia_recorrida += distancia_actual
    return distancia_recorrida

In [10]:
def main_exhaustivo(tabla_distancias, cantidad_ciudades):

    distancia_minima = 9999999999
    t0 = time.perf_counter()
    recorridos_posibles = generar_recorridos_posibles(cantidad_ciudades)

    for recorrido in recorridos_posibles:
        distancia_recorrida = calcular_distancia(tabla_distancias, recorrido)
        if distancia_recorrida < distancia_minima:
            distancia_minima = distancia_recorrida
            recorrido_minimo = recorrido

    t1 = time.perf_counter()
    tiempo_ejecucion = t1 - t0

    return recorrido_minimo, distancia, tiempo_ejecucion

recorrido mínimo: [0, 1, 2, 3, 4, 0]
disntancia: 2501.0


In [54]:
def main_heuristicoB(tabla_distancias):
    t0 = time.perf_counter()
    menor_distancia = 99999999999

    for i in range(len(tabla_distancias)):
        recorrido_actual = []
        recorrido_actual.append(i)
        distancia_recorrida = 0
        tabla_auxiliar = tabla_distancias[i].copy() #inicializar tabla auxiliar

        while len(recorrido_actual) < len(tabla_distancias): 
            distancia_mas_cercana = tabla_auxiliar.min()
            id_capital_sig = tabla_auxiliar.idxmin()
            
            if id_capital_sig in recorrido_actual:  # si verdadero, esa capital ya se visitó y por lo tanto se elimina de tabla auxiliar
                tabla_auxiliar = tabla_auxiliar.drop(id_capital_sig) 
            else:
                distancia_recorrida += distancia_mas_cercana  
                recorrido_actual.append(id_capital_sig)
                tabla_auxiliar = tabla_distancias[id_capital_sig].copy() # reinicio la tabla auxiliar para que tenga todas las filas
        
        recorrido_actual.append(i) #vuelvo a la ciudad de inicio
        distancia_recorrida += tabla_distancias[recorrido_actual[-1]][recorrido_actual[-2]] # sumo la distancia de vuelta a casa

        if distancia_recorrida < menor_distancia:
            menor_distancia = distancia_recorrida
            menor_recorrido = recorrido_actual

    tf = time.perf_counter()
    tiempo_ejecucion = tf - t0
    return menor_recorrido, menor_distancia, tiempo_ejecucion